In [19]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
from statsmodels.sandbox.regression import gmm

def prepare_instruments(data, num_lags, return_col_names):
    for lag in range(1, num_lags + 1):
        data[f'c_growth_lag{lag}'] = data['c_growth'].shift(lag)
        for col in return_col_names:
            data[f'{col}_lag{lag}'] = data[col].shift(lag)
    return data

# Define the moment function
def moment_consumption1(params, exog):
    delta, alpha = params
    c_growth, *assets  = exog.T  # unwrap iterable (ndarray)

    # Moment condition without instrument
    err = np. zeros((len(c_growth),))
    for asset in assets:
        err += 1 - delta * (1 + asset) * np.power(c_growth+1 , -alpha)
    return (err)/len(assets)

def estimate_gmm(data, num_lags, return_col_names):
    dta = prepare_instruments(data, num_lags, return_col_names)

    dta_clean = dta.dropna()

    endog_df = dta_clean[['c_growth']+[col for col in return_col_names]]
    exog_df = endog_df
    instrument_columns = ['const'] + [f'{col}_lag{i}' for i in range(1, num_lags + 1) for col in return_col_names]
    instrument_df = dta_clean[instrument_columns]

    endog, exog, instrument = map(np.asarray, [endog_df, exog_df, instrument_df])

    endog1 = np.zeros(exog.shape[0])
    
    mod1 = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=(num_lags+1)*(len(return_col_names)+1)+1)
    w0inv = np.dot(instrument.T, instrument) / len(endog1)
    I = np.eye(instrument.shape[1])  #identity matrix
    res1 = mod1.fit([1, -1], maxiter=10, inv_weights=I)

    
    j_stat, p_value, df = res1.jtest()# Degrees of freedom for the J-test

    
    return res1.params, res1.bse, (j_stat, p_value, df)

def consolidated_results(dta, lags_to_test, asset_names):
    # Estimate GMM for each number of lags and store the results
    results = [estimate_gmm(dta, num_lags, asset_names) for num_lags in lags_to_test]

    # Create DataFrames to display the results
    params_df = pd.DataFrame([res[0] for res in results], columns=['discount', 'CRRA'], index=lags_to_test)
    bse_df = pd.DataFrame([res[1] for res in results], columns=['SE_discount', 'SE_CRRA'], index=lags_to_test)

    jtest_df = pd.DataFrame([{'J-stat': res[2][0], 'p-value': res[2][1], 'df': res[2][2]} for res in results], index=lags_to_test)

    # Combine the DataFrames to display the results
    result_df = pd.concat([params_df, bse_df, jtest_df], axis=1)
    result_df.index.name = 'lags'

    # Print the results
    print(result_df)

# Read the CSV file and parse dates in the first column
file_path = 'spx_data.csv'
dta = pd.read_csv(file_path, parse_dates=[0])



# George Note: for now please change real_SP_return into "real"+industries if you want to run industry portfolio
    
# Calculate additional columns needed for the analysis
dta['c_growth'] = dta['real_pc_consumption'] / dta['real_pc_consumption'].shift(1)-1

dta['sp_return'] = dta['real_SP_return']
dta['rf1yr'] = dta['Real_1yr'] -1
dta['const'] = 1



# List of lags to be tested
lags_to_test = [ 1,2,4,6]
asset_names = ['sp_return', 'rf1yr']

consolidated_results(dta, lags_to_test, asset_names)

119
Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 6
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: 0.060751
         Iterations: 5
         Function evaluations: 8
         Gradient evaluations: 8
Optimization terminated successfully.
         Current function value: 0.058100
         Iterations: 4
         Function evaluations: 7
         Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 0.057185
         Iterations: 3
         Function evaluations: 6
         Gradient evaluations: 6
Optimization terminated successfully.
         Current function value: 0.057114
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.057109
         Iterations: 1
         Function evaluations: 3
         Grad

In [20]:
# Read the CSV file and parse dates in the first column
file_path = 'industry_data.csv'
dta = pd.read_csv(file_path, parse_dates=[0])

# Calculate additional columns needed for the analysis
dta['c_growth'] = dta['real_pc_consumption'] / dta['real_pc_consumption'].shift(1)-1

dta['sp_return'] = dta['real_SP_return']
dta['rf1yr'] = dta['Real_1yr'] -1
dta['const'] = 1

industries = ['Cnsmr', 'Manuf', "HiTec", 'Hlth', 'Other']
dta['cpi_change'] = dta['CPI'] / dta['CPI'].shift(1)-1
for industry in industries:
    dta[industry+"_r"] = dta[industry]/100- dta['cpi_change']
    
# List of lags to be tested
lags_to_test = [ 1,2,4,6]
asset_names = ['sp_return', 'rf1yr', 'Cnsmr_r', 'Manuf_r', 'HiTec_r', 'Hlth_r', 'Other_r']

consolidated_results(dta, lags_to_test, asset_names)

81
Optimization terminated successfully.
         Current function value: 0.000031
         Iterations: 5
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.046069
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.045911
         Iterations: 2
         Function evaluations: 6
         Gradient evaluations: 6
Optimization terminated successfully.
         Current function value: 0.045915
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.045915
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
80
Optimization terminated successfully.
         Current function value: 0.000535
         Iterations: 6
         Function evaluations: 12
         G